In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("data_US_Europe.csv")
tsValues = data['EXUSEU']
import math
tsValuesLog = [math.log(ele) for ele in tsValues]

In [2]:
PredictionOutputMAE = pd.DataFrame(index=['monthly','3_month','6_month','12_month'])
PredictionOutputRMSE = pd.DataFrame(index=['monthly','3_month','6_month','12_month'])
startPrediction = 100
totalData = len(tsValuesLog)

In [3]:
def mean_error(pred,real):
    return np.mean([pred[i]-real[i] for i in range(len(pred))])
def mean_absolute_error(pred,real):
    return np.mean([np.abs(pred[i]-real[i]) for i in range(len(pred))])
def root_mean_square_error(pred,real):
    return np.sqrt(np.mean([pow(pred[i]-real[i],2) for i in range(len(pred))]))

In [5]:
from statsmodels.tsa.arima_model import ARMA
mae_ = []
rmse_ = []
p=1
q=1
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        try:
            model = ARMA(tsValuesLog_d_1[i:i+startPrediction],order=(p,q))
            result = model.fit()
            pred.append(sum(result.predict(start=startPrediction,end=startPrediction+l-1))+tsValuesLog[i+startPrediction-1])
        except ValueError:
            pred.append(tsValuesLog[i+startPrediction-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_Diff_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_Diff_'+str(p)+'_'+str(q)] = rmse_

In [9]:
for [p,q] in [[1,2]]:
    print(p,q)

1 2


In [11]:
from statsmodels.tsa.arima_model import ARMA
mae_ = []
rmse_ = []
test_queue = [[1,1],[0,1],[1,0]]
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        single_pred = []
        for [p,q] in test_queue:
            try:
                model = ARMA(tsValuesLog_d_1[i:i+startPrediction],order=(p,q))
                result = model.fit()
                single_pred.append(sum(result.predict(start=startPrediction,end=startPrediction+l-1))+tsValuesLog[i+startPrediction-1])
            except ValueError:
                sing_pred.append(tsValuesLog[i+startPrediction-1])
        pred.append(np.mean(single_pred))
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_Diff_Mean'] = mae_
PredictionOutputRMSE['AR_Diff_Mean'] = rmse_

In [12]:
PredictionOutputRMSE

,AR_Diff_1_1,AR_Diff_Mean
monthly,0.017768,0.017464
3_month,0.039007,0.038765
6_month,0.064449,0.064286
12_month,0.108526,0.108422


In [13]:
PredictionOutputMAE

,AR_Diff_1_1,AR_Diff_Mean
monthly,0.013399,0.013121
3_month,0.029568,0.029352
6_month,0.048988,0.048937
12_month,0.082072,0.081848


In [4]:
#benchmark
mae_ = []
rmse_ = []
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        pred.append(tsValuesLog[startPrediction-1+i])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
PredictionOutputMAE['RandomWalk'] = mae_
PredictionOutputRMSE['RandomWalk'] = rmse_

In [5]:
PredictionOutputMAE

,RandomWalk
monthly,0.013241
3_month,0.029604
6_month,0.049440
12_month,0.075470


In [6]:
startPrediction=100

In [7]:
#test performance random walk with drift

mae_ = []
rmse_ = []
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        monthly_change = [tsValuesLog[j]-tsValuesLog[j-l] for j in range(i+l,i+startPrediction)]
        mean_monthly_change = np.mean(monthly_change)
        pred.append(tsValuesLog[startPrediction+i-1]+mean_monthly_change)
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['RandomWalkWithDrift'] = mae_
PredictionOutputRMSE['RandomWalkWithDrift'] = rmse_

In [8]:
PredictionOutputMAE

,RandomWalk,RandomWalkWithDrift
monthly,0.013241,0.013489
3_month,0.029604,0.029912
6_month,0.049440,0.050560
12_month,0.075470,0.081845


In [9]:
len(pred)

173

In [10]:
from statsmodels.tsa.arima_model import ARMA
#out-of-sample-test
mae_ = []
rmse_ = []
p=1
q=1
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        model = ARMA(tsValuesLog[i:i+startPrediction],order=(p,q))
        result = model.fit()
        pred.append(result.predict(start=startPrediction,end=startPrediction+l-1)[l-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_'+str(p)+'_'+str(q)] = rmse_

In [11]:
mae_ = []
rmse_ = []
p=2
q=0
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        model = ARMA(tsValuesLog[i:i+startPrediction],order=(p,q))
        result = model.fit()
        pred.append(result.predict(start=startPrediction,end=startPrediction+l-1)[l-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_'+str(p)+'_'+str(q)] = rmse_

In [12]:
mae_ = []
rmse_ = []
p=8
q=0
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        model = ARMA(tsValuesLog[i:i+startPrediction],order=(p,q))
        result = model.fit()
        pred.append(result.predict(start=startPrediction,end=startPrediction+l-1)[l-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_'+str(p)+'_'+str(q)] = rmse_

In [18]:
PredictionOutputMAE

,RandomWalk,RandomWalkWithDrift,AR_1_1,AR_2_0,AR_8_0,AR_Diff_1_0,AR_Diff_0_1
monthly,0.013241,0.013489,0.014220,0.013849,0.014635,0.012805,0.013196
3_month,0.029604,0.029912,0.033857,0.034844,0.036246,0.029276,0.029341
6_month,0.049440,0.050560,0.058562,0.061914,0.066017,0.048782,0.049083
12_month,0.075470,0.081845,0.092737,0.099036,0.110928,0.081626,0.081846


In [19]:
PredictionOutputRMSE

,RandomWalk,RandomWalkWithDrift,AR_1_1,AR_2_0,AR_8_0,AR_Diff_1_0,AR_Diff_0_1
monthly,0.017759,0.017836,0.018958,0.018617,0.019338,0.017093,0.017587
3_month,0.038925,0.039683,0.044698,0.045948,0.047205,0.038501,0.038838
6_month,0.062944,0.065691,0.072856,0.077019,0.085029,0.064093,0.064353
12_month,0.099058,0.107228,0.110591,0.116800,0.132660,0.108392,0.108377


In [4]:
tsValuesLog_d_1 = [0]
tsValuesLog_d_1 += [tsValuesLog[i+1]-tsValuesLog[i] for i in range(totalData-1)]

In [16]:
mae_ = []
rmse_ = []
p=1
q=0
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        model = ARMA(tsValuesLog_d_1[i:i+startPrediction],order=(p,q))
        result = model.fit()
        pred.append(sum(result.predict(start=startPrediction,end=startPrediction+l-1))+tsValuesLog[i+startPrediction-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_Diff_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_Diff_'+str(p)+'_'+str(q)] = rmse_

In [17]:
mae_ = []
rmse_ = []
p=0
q=1
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        model = ARMA(tsValuesLog_d_1[i:i+startPrediction],order=(p,q))
        result = model.fit()
        pred.append(sum(result.predict(start=startPrediction,end=startPrediction+l-1))+tsValuesLog[i+startPrediction-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:])) 
PredictionOutputMAE['AR_Diff_'+str(p)+'_'+str(q)] = mae_
PredictionOutputRMSE['AR_Diff_'+str(p)+'_'+str(q)] = rmse_

In [39]:
#Unobserved model
import statsmodels.api as sm

class UnobservedComponentModel:
    def __init__(self):
        self.model_name = "UnobservedComponentModel"
        return

    def fit(self, ts):
        unrestricted_model = {
            'level': 'local linear trend', 'cycle': False , 'seasonal': 12
        }

        model = sm.tsa.UnobservedComponents(endog=ts, **unrestricted_model)
        self.trained_model = model.fit()
        return self

    def predict(self, next_n_prediction):
        prediction = self.trained_model.forecast(steps=next_n_prediction)
        return prediction

In [40]:
mae_ = []
rmse_ = []
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-(l-1)):
        model = UnobservedComponentModel()
        model.fit(tsValuesLog[i:i+startPrediction])
        pred.append(model.predict(l)[l-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))

C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check

In [37]:
PredictionOutputMAE["UnobservedComponentModel"] = mae_
PredictionOutputRMSE["UnobservedComponentModel"] = rmse_

In [41]:
PredictionOutputMAE["UnobservedComponentModel_With_Season"] = mae_
PredictionOutputRMSE["UnobservedComponentModel_With_Season"] = rmse_

In [42]:
PredictionOutputMAE

,RandomWalk,RandomWalkWithDrift,AR_1_1,AR_2_0,AR_8_0,AR_Diff_1_0,AR_Diff_0_1,UnobservedComponentModel,UnobservedComponentModel_With_Season
monthly,0.013241,0.013489,0.014220,0.013849,0.014635,0.012805,0.013196,0.013811,0.015564
3_month,0.029604,0.029912,0.033857,0.034844,0.036246,0.029276,0.029341,0.034197,0.035016
6_month,0.049440,0.050560,0.058562,0.061914,0.066017,0.048782,0.049083,0.058489,0.056931
12_month,0.075470,0.081845,0.092737,0.099036,0.110928,0.081626,0.081846,0.113327,0.105037


In [43]:
PredictionOutputRMSE

,RandomWalk,RandomWalkWithDrift,AR_1_1,AR_2_0,AR_8_0,AR_Diff_1_0,AR_Diff_0_1,UnobservedComponentModel,UnobservedComponentModel_With_Season
monthly,0.017759,0.017836,0.018958,0.018617,0.019338,0.017093,0.017587,0.018318,0.020325
3_month,0.038925,0.039683,0.044698,0.045948,0.047205,0.038501,0.038838,0.044726,0.046606
6_month,0.062944,0.065691,0.072856,0.077019,0.085029,0.064093,0.064353,0.078753,0.077625
12_month,0.099058,0.107228,0.110591,0.116800,0.132660,0.108392,0.108377,0.145786,0.138788


In [44]:
PredictionOutputMAE.to_csv("EXUSEU_MAE.csv")

In [45]:
PredictionOutputRMSE.to_csv("EXUSEU_RMSE.csv")